# **課題 9: 干ばつモニタリングと作付状況評価（農業応用）**

**目的**
特定の農業地域（例：日本の主要な穀倉地帯）において、**干ばつの兆候を早期に検知**し、さらに**作物の健康状態（作付状況）を評価**するための分析パイプラインを構築します。

**対象地域・期間**
- 地域 (AOI): 茨城県南西部（米や野菜の栽培が盛んな地域。任意の矩形ポリゴンを設定してください）。

- 期間: 2024年の生育期間（例：5月〜8月）。

## **実行ステップとコードの骨子**

### **ステップ 1: 標準化植生指数（NDVI）の時系列解析**
作物の健康状態を監視するための基本的な指標であるNDVIの時系列データを作成します。"

#### 1. **Sentinel-2 Collection を読み込みます（$10 \text{ m}$解像度で詳細な農地分析に適しています）。**

In [ ]:
import ee
import geemap.foliumap as geemap

In [ ]:
ee.Authenticate(auth_mode='notebook')

In [ ]:
ee.Initialize(project='earth-change-analysis')

**県南地域**
- 土浦市: 緯度 36.0781, 経度 140.2014
- つくば市: 緯度 36.0838, 経度 140.0760
- 牛久市: 緯度 35.9792, 経度 140.1433
- 龍ケ崎市: 緯度 35.9083, 経度 140.1783
  
**県西地域**
- 筑西市: 緯度 36.3047, 経度 139.9783
- 結城市: 緯度 36.3000, 経度 139.8833
- 下妻市: 緯度 36.1833, 経度 139.9667

In [ ]:
ibaraki_sw = ee.Geometry.Rectangle(
  139.80, 35.80, # 最小経度、最小緯度 (南西)
  140.30, 36.32  # 最大経度、最大緯度 (北東)
)

map =  geemap.Map()

# AOIをマップに追加して表示
map.centerObject(ibaraki_sw, 10)  # AOIを中心にマップを移動し、ズームレベルを設定
map.add_layer(ibaraki_sw, {"color": 'FF0000'}, 'AOI: Ibaraki Southwest Agriculture')
map

In [ ]:
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(ibaraki_sw)\
    .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 20)   # 雲被覆率5%未満

s2_2024_collection = s2_collection.filterDate('2024-05-01', '2024-08-31')

#### 2. **コレクション全体に $\text{NDVI}$ 算出関数を適用し、$\text{NDVI}$ の時系列を作成します。**
  
   （ヒント: $\text{B8}$ (NIR) と $\text{B4}$ (Red) を使用。）

In [ ]:
# NDVIを計算する関数の定義
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# コレクション全体に適用して時系列を作成
ndvi_collection_2024 = s2_2024_collection.map(addNDVI)

#### 3. **時系列から、期間全体の**$\text{NDVI}$ 中央値合成画像**を作成し、地図上に表示します。**

In [ ]:
ndvi_2024_median = ndvi_collection_2024.select('NDVI').median()

# NDVI画像を調査範囲で切り抜く（この1行を追加、またはadd_layer内で実行）
ndvi_2024_median_clipped = ndvi_2024_median.clip(ibaraki_sw)

vis_params = {
    "min": 0.0,
    "max": 1.0,
    "palette": ["white", "blue", "yellow", "green"],
    "opacity": 0.7
}

# 矩形領域の中心座標を計算して地図の中心に設定
center_coords = ibaraki_sw.centroid().coordinates().getInfo()
map_center = [center_coords[1], center_coords[0]]  # [lat, lon] 形式に変換

map = geemap.Map(center=map_center, zoom=10)
map.add_basemap("OpenStreetMap")    
map.add_basemap("HYBRID")   
map.add_layer(ibaraki_sw, {"color": "red"}, "Rsearch Area")
map.add_layer(ndvi_2024_median_clipped, vis_params, 'NDVI')
#  NDVI用のカラーバー（右下に配置）
map.add_colorbar(vis_params, label="NDVI", layer_name="NDVI")

# map.to_html("ndvi_ibaraki_southwest_2024.html") 
map

### **ステップ 2: 干ばつ指標（$\text{VCI}$）の算出**

干ばつの影響を定量化するために、植生被覆指数（$\text{VCI}$: $\text{Vegetation Condition Index}$）を算出します。$\text{VCI}$ は、特定の時期の $\text{NDVI}$ が、その場所の**歴史的な $\text{NDVI}$ の最大値と最小値**と比較してどの程度の状態にあるかを示します。

$$\text{VCI} = \frac{\text{NDVI} - \text{NDVI}_{\min}}{\text{NDVI}_{\max} - \text{NDVI}_{\min}} \times 100$$

#### 1. **歴史的な $\text{NDVI}$ 統計量の算出:**

過去5年間（例: 2019年〜2023年）の同じ生育期間（5月〜8月）の $\text{NDVI}$ コレクションを作成します。このコレクションから、ピクセルごとの**$\text{NDVI}$ の最小値 ($\text{NDVI}_{\min}$) と最大値 ($\text{NDVI}_{\max}$)** を算出します。

In [ ]:
# 1. 過去5年・5月〜8月でフィルタリング
historical_col = s2_collection \
    .filter(ee.Filter.calendarRange(2019, 2023, 'year')) \
    .filter(ee.Filter.calendarRange(5, 8, 'month')) \
    .map(addNDVI) # NDVI計算関数を適用

# 2. ピクセルごとに最大・最小を算出
ndvi_max = historical_col.select('NDVI').max()
ndvi_min = historical_col.select('NDVI').min()

#### 2. **$\text{VCI}$ の算出:**

1で得た統計量と、2024年の $\text{NDVI}$ 中央値画像 ($\text{NDVI}_{2024}$) を用いて $\text{VCI}$ を算出します。$\text{VCI}$ は $0$（極度の干ばつ）から $100$（非常に良い状態）の範囲で可視化します。

In [ ]:
# VCIの計算
vci = ndvi_2024_median.subtract(ndvi_min) \
    .divide(ndvi_max.subtract(ndvi_min)) \
    .multiply(100)\
    .rename('VCI')


In [ ]:
# そのオブジェクトのGEEタイプ名を返す
print(vci.name())

In [ ]:
vci

In [ ]:
# 1. 領域内の平均値を計算（reduceRegionを使用）
stats = vci.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=ibaraki_sw,
    scale=10,
    maxPixels=1e9
).getInfo()

# 2. 結果を取り出す（'VCI' バンドの数値を取得）
vci_value = stats.get('VCI')

# 3. None（空）でないことを確認して表示
if vci_value is not None:
    print(f"調査範囲の平均VCI値: {vci_value:.2f}")
else:
    print("VCI値を計算できませんでした。データが空か、範囲外です。")

#### 3. **$\text{VCI}$ 画像を地図に追加します。**

In [ ]:
# 表示パラメータの設定
vci_vis = {
    'min': 0,
    'max': 100,
    'palette': ['#e74c3c', '#f1c40f', '#2ecc71'], # 赤 → 黄 → 緑
    'opacity': 0.7
}

# 地図の作成
map = geemap.Map(center=map_center, zoom=10)
map.add_basemap("OpenStreetMap")     

# VCIをクリップして追加
map.add_layer(vci.clip(ibaraki_sw), vci_vis, 'VCI 2024')
map.add_colorbar(vci_vis, label="VCI (0-100)", layer_name="VCI 2024") # layer_nameを指定
# map.to_html("vci_ibaraki_southwest_2024.html")  
map

### **ステップ 3:**

収量リスクエリアの特定とレポート$VCI$ の値に基づき、干ばつによる収量リスクが高いエリアを特定し、面積を計算します。

#### **1. リスクエリアの定義:** $\text{VCI}$ スコアが $\mathbf{35}$ **未満**のエリアを「**干ばつ高リスクエリア**」と定義します。

In [ ]:
vci_low_2024 = vci.lt(35)

#### 2. この高リスクエリアのバイナリマスクを作成します。

>この文章は、解析の最終段階において「ある特定の条件（今回の場合はVCIが低いなど）に合致する場所だけを『1』、それ以外を『0』とした白黒の画像（デジタル地図）を作る」という指示です。

In [ ]:
high_risk_mask = vci_low_2024.updateMask(vci_low_2024)  
pixel_area =  ee.Image.pixelArea()  
high_risk_area_image = high_risk_mask.multiply(pixel_area).rename('HighRiskArea')   
area_stats = high_risk_area_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=ibaraki_sw,
    scale=10,
    maxPixels=1e9
).getInfo() 

area_m2 = area_stats.get('HighRiskArea', 0)
area_km2 = area_m2 / 1e6    
print(f"調査範囲内の高リスク地域の面積: {area_km2:.2f} km²")    

```python
# 1. ピクセル面積画像を作成（単位：m²）
pixel_area = ee.Image.pixelArea()

# 2. 緑化ポテンシャル画像（0/1）にピクセル面積を掛ける
area_image = greeningPotential_2023.multiply(pixel_area).rename('Green Potential Area')

# The default projection is WGS84 with 1-degree scale.
display('Pixel area default projection', area_image.projection())

# 3. 合計面積を reduceRegion で計算（例：京都市のAOI内）
area_stats = area_image.reduceRegion(
    reducer = ee.Reducer.sum(),
    geometry = kyoto_aoi,
    scale = 10,  # 解像度（Landsatなら30m、Sentinel-2なら10m）
    maxPixels=1e9
)

# 2. getInfo() で Python の dict に変換
area_dict = area_stats.getInfo()

# 3. 値を取り出して float にしてから計算
area_m2 = area_dict['Green Potential Area']  # ← 'Green Potential Area' は reduceRegion のキー名
area_km2 = area_m2 / 1e6

# 4. 表示
print(f"緑化ポテンシャルエリアの面積: {area_m2:,.0f} m²（約 {area_km2:,.2f} km²）")
```